In [8]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, HBox, VBox
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Параметры сетки
M, N = 50, 50  # Число узлов по x и y
Lx, Ly = 1.0, 1.0  # Размеры области
hx, hy = Lx / (M - 1), Ly / (N - 1)  # Шаги сетки

# Аналитические решения для проверки точности
def phi_analytical(task, x, y):
    if task == 1:
        return x * (1 - x) + y * (1 - y)
    elif task == 2:
        return x**2 * (1 - x)**2 + y**2 * (1 - y)**2
    elif task == 3:
        return np.sin(2 * np.pi * x) * np.sin(2 * np.pi * y)
    elif task == 4:
        return np.sin(4 * np.pi * x) * np.sin(4 * np.pi * y)

# Определение задач (источников)
def rho_task(task, x, y):
    if task == 1:
        return 1 / np.pi  # -∇²φ = 0 + const
    elif task == 2:
        return (1 + 3 * (x + y) - 3 * (x**2 + y**2)) / np.pi  # -∇²φ = выражение
    elif task == 3:
        return 8 * np.pi * np.sin(2 * np.pi * x) * np.sin(2 * np.pi * y)  # -∇²φ = -8π²φ
    elif task == 4:
        return 32 * np.pi**2 * np.sin(4 * np.pi * x) * np.sin(4 * np.pi * y)  # -∇²φ = -32π²φ

def phi_boundary_task(task, x, y):
    if task == 1:
        if y == 0:
            return x * (1 - x)  # Нижняя граница
        elif y == Ly:
            return x * (1 - x)  # Верхняя граница
        elif x == 0:
            return y * (1 - y)  # Левая граница
        elif x == Lx:
            return y * (1 - y)  # Правая граница
    elif task == 2:
        if y == 0 or y == Ly:
            return x**2 * (1 - x)**2
        elif x == 0 or x == Lx:
            return y**2 * (1 - y)**2
    elif task == 3 or task == 4:
        return 0  # Нулевые граничные условия
    return 0

# Инициализация сетки
def initialize_grid(task):
    phi = np.zeros((M, N))
    rho_grid = np.zeros((M, N))
    x = np.linspace(0, Lx, M)
    y = np.linspace(0, Ly, N)
    
    # Заполнение источника
    for i in range(M):
        for j in range(N):
            rho_grid[i, j] = rho_task(task, x[i], y[j])
    
    # Применение граничных условий
    for i in range(M):
        phi[i, 0] = phi_boundary_task(task, x[i], 0)
        phi[i, -1] = phi_boundary_task(task, x[i], Ly)
    for j in range(N):
        phi[0, j] = phi_boundary_task(task, 0, y[j])
        phi[-1, j] = phi_boundary_task(task, Lx, y[j])
    
    return phi.copy(), rho_grid, x, y

# Метод последовательной верхней релаксации (МПВР)
def solve_mpvr(task, omega=1.5, tol=1e-6, max_iter=10000):
    phi, rho_grid, x, y = initialize_grid(task)
    start_time = time.time()
    
    for iteration in range(max_iter):
        phi_old = phi.copy()
        for i in range(1, M - 1):
            for j in range(1, N - 1):
                phi[i, j] = (1 - omega) * phi[i, j] + omega / (2 / hx**2 + 2 / hy**2) * (
                    (phi[i + 1, j] + phi[i - 1, j]) / hx**2 +
                    (phi[i, j + 1] + phi[i, j - 1]) / hy**2 +
                    4 * np.pi * rho_grid[i, j]
                )
        
        # Проверка сходимости
        error = np.linalg.norm(phi - phi_old, ord=np.inf)
        if error < tol:
            break
    
    elapsed_time = time.time() - start_time
    
    # Вычисление ошибки относительно аналитического решения
    phi_exact = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            phi_exact[i, j] = phi_analytical(task, x[i], y[j])
    
    exact_error = np.linalg.norm(phi - phi_exact, ord=np.inf)
    
    return phi, x, y, iteration+1, elapsed_time, exact_error

# Метод простых итераций (МПИ) / Метод Якоби
def solve_mpi(task, tau=0.1, tol=1e-6, max_iter=10000):
    phi, rho_grid, x, y = initialize_grid(task)
    start_time = time.time()
    
    for iteration in range(max_iter):
        phi_old = phi.copy()
        for i in range(1, M - 1):
            for j in range(1, N - 1):
                phi[i, j] = (phi_old[i + 1, j] + phi_old[i - 1, j] + 
                            phi_old[i, j + 1] + phi_old[i, j - 1]) / 4 + tau * rho_grid[i, j]
                            
        # Проверка сходимости
        error = np.linalg.norm(phi - phi_old, ord=np.inf)
        if error < tol:
            break
    
    elapsed_time = time.time() - start_time
    
    # Вычисление ошибки относительно аналитического решения
    phi_exact = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            phi_exact[i, j] = phi_analytical(task, x[i], y[j])
    
    exact_error = np.linalg.norm(phi - phi_exact, ord=np.inf)
    
    return phi, x, y, iteration+1, elapsed_time, exact_error

# Метод стабилизирующей поправки (МСП)
def solve_msp(task, tau=0.1, tol=1e-6, max_iter=10000):
    phi, rho_grid, x, y = initialize_grid(task)
    start_time = time.time()
    
    for iteration in range(max_iter):
        phi_old = phi.copy()
        for i in range(1, M - 1):
            for j in range(1, N - 1):
                phi[i, j] = (phi[i + 1, j] + phi[i - 1, j] + 
                            phi[i, j + 1] + phi[i, j - 1]) / 4 + tau * (rho_grid[i, j] - 
                            (4*phi_old[i, j] - phi_old[i + 1, j] - phi_old[i - 1, j] - 
                             phi_old[i, j + 1] - phi_old[i, j - 1])/hx**2)
        
        # Проверка сходимости
        error = np.linalg.norm(phi - phi_old, ord=np.inf)
        if error < tol:
            break
    
    elapsed_time = time.time() - start_time
    
    # Вычисление ошибки относительно аналитического решения
    phi_exact = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            phi_exact[i, j] = phi_analytical(task, x[i], y[j])
    
    exact_error = np.linalg.norm(phi - phi_exact, ord=np.inf)
    
    return phi, x, y, iteration+1, elapsed_time, exact_error

# Метод двухкратного преобразования Фурье (ДПФ)
def solve_dpf(task):
    phi, rho_grid, x, y = initialize_grid(task)
    start_time = time.time()
    
    # Масштабирование сетки для работы с FFT
    I = np.arange(0, M)
    J = np.arange(0, N)
    kx = 2 * np.pi * I / Lx
    ky = 2 * np.pi * J / Ly
    KX, KY = np.meshgrid(kx, ky, indexing="ij")
    
    # Быстрое преобразование Фурье плотности заряда
    rho_hat = np.fft.fft2(rho_grid)
    
    # Решение уравнения Пуассона в частотном пространстве
    denominator = KX**2 + KY**2
    # Избегаем деления на ноль для нулевой частоты
    denominator[0, 0] = 1.0
    phi_hat = rho_hat / denominator
    phi_hat[0, 0] = 0.0  # Постоянная интегрирования
    
    # Обратное преобразование Фурье для получения решения
    phi_interior = np.real(np.fft.ifft2(phi_hat))
    
    # Необходимо применить граничные условия
    for i in range(M):
        phi[i, 0] = phi_boundary_task(task, x[i], 0)
        phi[i, -1] = phi_boundary_task(task, x[i], Ly)
    for j in range(N):
        phi[0, j] = phi_boundary_task(task, 0, y[j])
        phi[-1, j] = phi_boundary_task(task, Lx, y[j])
    
    # Копируем внутреннюю часть решения
    phi[1:-1, 1:-1] = phi_interior[1:-1, 1:-1]
    
    elapsed_time = time.time() - start_time
    
    # Вычисление ошибки относительно аналитического решения
    phi_exact = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            phi_exact[i, j] = phi_analytical(task, x[i], y[j])
    
    exact_error = np.linalg.norm(phi - phi_exact, ord=np.inf)
    
    return phi, x, y, 0, elapsed_time, exact_error  # 0 итераций для ДПФ

# Визуализация решения
def visualize_solution(phi, x, y, title, task=1, add_colorbar=True):
    X, Y = np.meshgrid(x, y, indexing="ij")
    fig, ax = plt.subplots(figsize=(6, 5))
    
    contour = ax.contourf(X, Y, phi, levels=50, cmap="viridis")
    if add_colorbar:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(contour, cax=cax, label="φ(x, y)")
    
    ax.set_title(title)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    
    return fig, ax

# Создание сравнительного графика
def compare_methods(task, omega=1.5, tau=0.1):
    methods = {
        "МПВР": lambda: solve_mpvr(task, omega),
        "МПИ": lambda: solve_mpi(task, tau),
        "МСП": lambda: solve_msp(task, tau),
        "ДПФ": lambda: solve_dpf(task)
    }
    
    results = {}
    
    # Создаем макет фигур 2x2
    fig = plt.figure(figsize=(16, 14))
    
    # Решаем задачу каждым методом и визуализируем
    for i, (name, solver) in enumerate(methods.items()):
        phi, x, y, iterations, elapsed_time, exact_error = solver()
        results[name] = {
            "phi": phi,
            "iterations": iterations,
            "time": elapsed_time,
            "error": exact_error
        }
        
        # Отображаем результат
        ax = fig.add_subplot(2, 2, i+1)
        X, Y = np.meshgrid(x, y, indexing="ij")
        contour = ax.contourf(X, Y, phi, levels=50, cmap="viridis")
        plt.colorbar(contour, ax=ax)
        ax.set_title(f"{name}: Итераций={iterations}, Время={elapsed_time:.4f} с, Ошибка={exact_error:.6f}")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
    
    # Отображение аналитического решения
    phi_exact = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            phi_exact[i, j] = phi_analytical(task, x[i], y[j])
    
    # Добавляем график разницы между аналитическим и численным решениями
    fig2 = plt.figure(figsize=(16, 8))
    for i, (name, result) in enumerate(results.items()):
        ax = fig2.add_subplot(1, len(methods), i+1)
        diff = np.abs(result["phi"] - phi_exact)
        X, Y = np.meshgrid(x, y, indexing="ij")
        contour = ax.contourf(X, Y, diff, levels=50, cmap="hot")
        plt.colorbar(contour, ax=ax)
        ax.set_title(f"Ошибка {name}: {result['error']:.6f}")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
    
    plt.tight_layout()
    plt.show()
    
    return results

# Интерактивность
def interactive_solver(task, method, omega=1.5, tau=0.1):
    if method == "МПВР":
        phi, x, y, iterations, elapsed_time, exact_error = solve_mpvr(task, omega)
    elif method == "МПИ":
        phi, x, y, iterations, elapsed_time, exact_error = solve_mpi(task, tau)
    elif method == "МСП":
        phi, x, y, iterations, elapsed_time, exact_error = solve_msp(task, tau)
    elif method == "ДПФ":
        phi, x, y, iterations, elapsed_time, exact_error = solve_dpf(task)
        
    visualize_solution(phi, x, y, 
                      f"Задача {task}, Метод {method}\nИтераций: {iterations}, Время: {elapsed_time:.4f} с, Ошибка: {exact_error:.6f}")
    
    # Рассчитываем точное решение для сравнения
    phi_exact = np.zeros((M, N))
    for i in range(M):
        for j in range(N):
            phi_exact[i, j] = phi_analytical(task, x[i], y[j])
    
    # Отображаем разницу между численным и аналитическим решениями
    diff = np.abs(phi - phi_exact)
    plt.figure(figsize=(8, 6))
    X, Y = np.meshgrid(x, y, indexing="ij")
    plt.contourf(X, Y, diff, levels=50, cmap="hot")
    plt.colorbar(label="Абсолютная ошибка")
    plt.title(f"Ошибка решения для Задачи {task}, Метод {method}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

# Виджет для сравнения всех методов на одной задаче
def compare_widget():
    compare_button = widgets.Button(description="Сравнить все методы")
    task_slider = widgets.IntSlider(min=1, max=4, step=1, value=1, description="Задача", style={'description_width': '100px'})
    omega_slider = widgets.FloatSlider(min=1.0, max=2.0, step=0.1, value=1.5, description="ω (для МПВР)", style={'description_width': '100px'})
    tau_slider = widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description="τ (для МПИ/МСП)", style={'description_width': '100px'})
    
    def on_compare(_):
        compare_methods(task_slider.value, omega_slider.value, tau_slider.value)
        
    compare_button.on_click(on_compare)
    return VBox([task_slider, omega_slider, tau_slider, compare_button])

# Основные виджеты
task_widget = widgets.IntSlider(min=1, max=4, step=1, value=1, description="Задача")
method_widget = widgets.RadioButtons(
    options=["МПВР", "МПИ", "МСП", "ДПФ"], 
    value="МПВР", 
    description="Метод", 
    style={'description_width': '100px'}
)
omega_widget = widgets.FloatSlider(min=1.0, max=2.0, step=0.1, value=1.5, description="ω (МПВР)", style={'description_width': '100px'})
tau_widget = widgets.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description="τ (МПИ/МСП)", style={'description_width': '100px'})

# Организуем все в одном интерфейсе
main_widget = interact(
    interactive_solver,
    task=task_widget,
    method=method_widget,
    omega=omega_widget,
    tau=tau_widget
)

# Добавляем панель сравнения
# compare_panel = compare_widget()
# display(widgets.HTML("<h3>Сравнение методов</h3>"))
# display(compare_panel)

interactive(children=(IntSlider(value=1, description='Задача', max=4, min=1), RadioButtons(description='Метод'…

# Ожидаемые результаты численного решения уравнения Пуассона

## Сравнение методов и их особенностей

### Метод последовательной верхней релаксации (МПВР)
- **Скорость сходимости**: Хорошая при оптимальном выборе параметра релаксации ω
- **Ожидаемое поведение**: 
  - При ω < 1 медленная сходимость
  - При ω ≈ 1.5-1.8 обычно наблюдается оптимальная скорость
  - При ω > 1.9 возможна нестабильность и расходимость
- **Число итераций**: Должно быть меньше, чем у МПИ (особенно при оптимальном ω)

### Метод простых итераций (МПИ)
- **Скорость сходимости**: Наиболее медленная среди всех итерационных методов
- **Чувствительность к τ**: При τ > 0.25 метод может стать нестабильным
- **Число итераций**: Наибольшее из всех методов

### Метод стабилизирующей поправки (МСП)
- **Скорость сходимости**: Лучше, чем у МПИ, но медленнее МПВР
- **Стабильность**: Более устойчив к выбору параметра τ
- **Преимущества**: Более эффективен для задач с большими градиентами

### Двукратное преобразование Фурье (ДПФ)
- **Скорость**: Мгновенное решение (не требует итераций)
- **Точность**: Высокая для периодических граничных условий
- **Ограничения**: Может давать погрешности для непериодических граничных условий

## Ожидаемые результаты по задачам

### Задача 1: ρ = 1/π (постоянный источник)
- **Аналитическое решение**: φ(x,y) = x(1-x) + y(1-y)
- **Форма решения**: Гладкая выпуклая поверхность с максимумом в центре
- **Эффективность методов**: 
  - МПВР должен быстро сходиться (10-50 итераций при ω ≈ 1.7)
  - ДПФ может иметь некоторые погрешности на границах из-за непериодичности

### Задача 2: ρ = (1+3(x+y)-3(x²+y²))/π
- **Аналитическое решение**: φ(x,y) = x²(1-x)² + y²(1-y)²
- **Форма решения**: Более сложная структура с четырьмя симметричными пиками
- **Эффективность методов**: 
  - Потребуется больше итераций для всех методов по сравнению с задачей 1
  - МСП может быть более эффективен из-за более сложного распределения

### Задача 3: ρ = 8π²sin(2πx)sin(2πy)
- **Аналитическое решение**: φ(x,y) = sin(2πx)sin(2πy)
- **Форма решения**: Периодическая структура с максимумом в центре и минимумами в углах
- **Эффективность методов**: 
  - ДПФ должен давать очень точное решение из-за периодичности
  - Итерационные методы потребуют больше итераций (50-200) из-за осцилляций

### Задача 4: ρ = 32π²sin(4πx)sin(4πy)
- **Аналитическое решение**: φ(x,y) = sin(4πx)sin(4πy)
- **Форма решения**: Более частые осцилляции (в два раза больше, чем в задаче 3)
- **Эффективность методов**: 
  - ДПФ будет наиболее эффективным
  - Итерационные методы существенно замедлятся (200-500 итераций)
  - МПВР будет требовать более точной настройки ω

## Сравнительные метрики

### Время выполнения (от быстрого к медленному)
1. ДПФ (миллисекунды)
2. МПВР (секунды, зависит от ω)
3. МСП (секунды, обычно дольше МПВР)
4. МПИ (секунды, самый медленный)

### Точность (от высокой к низкой)
- Для задач 1 и 2 (непериодические):
  1. МПВР с достаточным числом итераций
  2. МСП
  3. ДПФ (возможны погрешности из-за непериодичности)
  4. МПИ
- Для задач 3 и 4 (периодические):
  1. ДПФ
  2. МПВР
  3. МСП
  4. МПИ

### Эффективность при увеличении размера сетки
- ДПФ показывает наилучшее масштабирование O(N·log N)
- МПВР, МСП и МПИ требуют O(N²) операций на итерацию, где N - размер сетки

Запустив код и сравнив методы, вы увидите эти тенденции визуально, а также сможете количественно оценить точность и скорость каждого метода для разных задач.